| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb) | [Lattice Strain](./09_lattice_strain.ipynb) | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

# Transforming Geochemical Data


In [ ]:
import pyrolite.geochem
import pandas as pd
import numpy as np
from pyrolite.util.synthetic import normal_frame

pd.options.display.precision = 3  # smaller graphical outputs

df = (
    normal_frame(
        columns=["CaO", "MgO", "SiO2", "FeO", "Na2O", "Ni", "Ti", "La", "Lu", "Te"]
    )
    * 100
)  # synthetic data with a multivariate normal distribution in log-transformed space
df[["Ni", "Ti", "La", "Lu", "Te"]] *= 10
df["Sr87/Sr86"] = 0.0700 / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

----
### Using Indexers, Scaling

You can also use these indexers for assignment, where the dimensionality of the dataset doesn't change. While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [ ]:
from pyrolite.util.units import scale

els = (
    df.pyrochem.elements.copy()
)  # get a copy of just the elements from the dataframe, we'll then edit this version
els.pyrochem.elements *= scale("ppm", "wt%")

In [ ]:
df.pyrochem.elements, els.pyrochem.elements

---
### Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [ ]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [ ]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

----
<div class='alert alert-warning'> <font size="+1" color="black"><b> Checkpoint & Time Check</b><br>How are things going?</font></div>

----


### Compositional Data - Logratio Transformation

pyrolite includes a few functions for dealing with compositional data, at the heart of which are i) closure (i.e. everything sums to 100%) and ii) log-transforms to deal with
the compositional space. We don't quite have room in this workshop to get into compositional data anlaysis, but there's a bit more information in [the pyrolite documentation](https://pyrolite.readthedocs.io/en/master/examples/index.html#compositional-data-examples). 

The commonly used log-transformations include the Additive Log-Ratio (`ALR`), Centred Log-Ratio (`CLR`), and Isometric Log-Ratio (`ILR`). Let's have a look at one of the log-transforms, which can be accessed directly from your dataframes (via the `df.pyrocomp` API). A key thing to note here is that everything should start in the same units and sum to one if you want it to be able to be back-transformed! Note we're using `df.pyrochem.compositional` to extract the elements and oxides by leave other columns alone:

In [ ]:
scaled_df = df.copy()
scaled_df.pyrochem.elements *= scale("ppm", "wt%")
scaled_df.pyrochem.compositional = (
    scaled_df.pyrochem.compositional.pyrocomp.renormalise(scale=1)
)

In [ ]:
scaled_df.head()

In [ ]:
lr_df = scaled_df.pyrochem.compositional.pyrocomp.CLR()
lr_df.head()

In [ ]:
back_transformed = lr_df.pyrocomp.inverse_CLR()
back_transformed.head()

One of the key areas where these logratio transforms might be useful is in deriving statistical properties from your geochemical data, for example calculating a mean. There's a specific function dedicated to this aspect:

In [ ]:
scaled_df.pyrochem.compositional.pyrocomp.logratiomean()

### Compositional Data - Spherical Transformation

pyrolite also includes a spherical transformation, which is particularly useful for scenarios where zeros are valid, which is where logratio methods fall down (e.g. mineralogy):

In [ ]:
scaled_df.pyrochem.compositional.pyrocomp.sphere()

| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) | [CIPW](./08_CIPW_Norm.ipynb) | [Lattice Strain](./09_lattice_strain.ipynb) | **Extensions:** | [ML](./11_geochem_ML.ipynb) | [Spatial Data](./12_spatial_geochem.ipynb) |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |